In [1]:
from bintree import BinNode,BinTree,BinNodeUtil,BinTreeUtility

### 1.1 二叉搜索树模板

#### 一. 二叉搜索树的结构
1. 二叉搜索树处处满足顺序性 :   
  任一节点$r$的左(右)子树中, 所有节点(若存在)均不大于(不小于)$r$ 
2. 若按照"左子树<父节点<右子树"的顺序组织节点, 则该二叉搜索树的中序遍历序列单调非降  
3. 1. 二叉搜索树BST继承自前文介绍的二叉树类  
 [二叉树类](http://nbviewer.jupyter.org/github/lj72808up/datastruct/blob/master/bintree/1-BinaryTree.ipynb)

#### 二. BST关键码查找
1. BST查找  
  1. 从根节点开始, 按照关键码比较结果决定递归查找左子树或右子树  
  2. BST类的属性hot, 记录查找节点的父节点(该位置可作为后面插入节点到BST的接入位置)   
  3. BST查找, 一般将叶子节点的None孩子节点也看做"真实节点",便于递归  
2. 查找效率  
 BST的每一层, 算法至多访问1个节点, 只需常数时间. 然而不幸的是, 规模为n的BST, 其深度可能为n. 此时退化成一个有序列表, 此时的查找等效于顺序查找, 因此提高BST的搜索效率, 关键在于减小树的高度
 
#### 三. BST节点插入
1. 插入步骤  
  1. 首先, 利用BST查找算法, 找到插入位置及方向, 然后将新节点作为叶子节点插入  
  2. 从节点hot开始, 更新其父节点的高度  
  
#### 四. BST节点删除
节点删除的情况比较复杂, 需要分情况讨论待删除节点的状态, 找到能够替换掉该节点的节点    
首先找到待删除节点的位置
1. 待删除节点只有单分支  
 当待删除节点只有单分支, 将该待删除节点替换为该节点的孩子节点即可  
2. 待删除节点有双分支  
 当待删除节点有双分支, 为保证BST的顺序定义(左孩子<节点<右孩子), 该节点应被替换为中序遍历下的后继节点:   
   1. 即若该节点有右孩子, 则其后继为右子树的最左节点
   2. 若该节点无右孩子, 则其后继为一直向左上方提升,知道无法提升后的父节点k
   其右子树的最左节点. 因为右子树的最左节点必无左孩子, 所以删除该最左节点的情形同只有一个分支的情况
<img src='img/bstremove1.png' height='85%' width='85%'>

In [2]:
class BST(BinTree):
    def __init__(self):
        # 命中节点的父亲
        self.hot = None
        self.root = None
        
    def searchIn(self,node,data):
        '''从节点node开始查找关键码为data的节点
           hot指向其父节点'''
        if (node is None ) or (node.data == data):
            return node
        self.hot = node
        if data<node.data:
            return search(node.lc,data)
        if data>node.data:
            return search(node.rc,data)
        
    def search(self,data):
        '''从根节点开始,查找关键码为data的节点'''
        return self.searchIn(self.root,data)
    
    def insert(self,data):
        '''将关键码data插入BST树'''
        x = self.search(data)
        # 查找成功, 直接返回
        if x is not None:
            return x
        # 查找不成功, 新建节点, 
        new_x = BinNode(data,parent=self.hot)
        # 插入子孩子
        if data<self.hot.data:
            self.hot.lc = new_x
        if data>self.hot.data:
            self.hot.rc = new_x
        # 更新节点数, 数高
        self.size = self.size + 1
        self.update_height_above(self,new_x)
        return new_x
    
    def removeAt(self,node):
        '''删除节点:
           succ : 返回值,为实际被删除节点的接替者
           self.hot : 实际被删除节点的parent. 
                      单分支下hot为被删除节点的父节点
                      双分支下hot为右子树最左节点的parent 
        '''
        w = node
        succ = None # 始终指向被删除节点的接替者
        if not BinNodeUtil.has_lchild(node):
            succ = node.rc
        elif not BinNodeUtil.has_rchild(node):
            succ = node.lc
        else: # 两个分支都存在
            w = BinTreeUtility.succ(w)
            # 交换待删除节点和其后继节点的元素值后, 再删除这个后继节点
            tmp = node.data
            node.data = w.data
            w.data = tmp
            u = w.parent
            if u is node: # 后继节点是该节点的父亲,
                succ = w.rc
                u.rc = w.rc
            else:
                succ = w.rc
                u.lc = w.rc
        self.hot = w.parent
        if succ is not None:
            succ.parent = self.hot
        return succ
    
    def remove(self,data):
        x = self.search(data)
        if x is None:
            return False
        self.removeAt(data)
        self.size = self.size-1
        self.update_height_above(self.hot)
        return True

### 1.2 平衡二叉树的树型调整

#### 一. 为什么要调整平衡二叉树的树形结构
1. 如上模板实现, search(),insert(),delete()方法, 其时间复杂度均线性正比于二叉搜索树的高度,  
 最坏情况下, 二叉搜索树可能退化为列表, 其查找效率甚至可能退化为$O(n)$  
2. 树型机构不同的原因   
  同一组数据, 由于插入二叉搜索树的顺序不同, 导致生成的BST树形结构不同
  <img src='img/bst2.png' width='75%' height='75%'>

#### 二. 等价二叉搜索树
1. 何为等价二叉树  
 若2棵二叉搜索树的中序遍历相同, 则称他们彼此等价  
 如下图所示的2棵二叉搜索树是等价的.
 <img src='img/dengjiaerchasousuoshu.png' width='70%' height='70%'>
2. 等价二叉搜索树的性质  
 如上图所示, 等价二叉搜索树, 各节点的垂直高度可能有所不同, 但水平次序完全一致. 即"上下可变,左右不乱"

#### 三. AVL树
1. 平衡因子  
 任意节点v的平衡因子, 定义为"其左右子树的高度差", 即$$balFac(v) = height\left( lc(v) \right) -height\left( rc(v) \right) $$[注]:空树高度为-1,单节点子树高度为0
2. AVL树的限制条件  
 AVL树, 即平衡因子受限的二叉搜索树, 其各节点平衡因子的绝对值不超过1
3. 为了判断平衡因子, 增加工具类`BalancedUtil`

In [ ]:
class AVL(BST):
    def insert(self,data):
        
    def remove(self,data):
        

In [ ]:
class BalancedUtil(object):
    @staticmethod
    def Balanced(node):
        '''理想平衡条件'''
        return BinNodeUtil.stature(node.lc) == BinNodeUtil.stature(node.rc)
    @staticmethod
    def BalFac(node):
        '''平衡因子'''
        return BinNodeUtil.stature(node.lc) - BinNodeUtil.stature(node.rc)
    @staticmethod
    def AvlBalanced(node):
        '''AVL平衡条件'''
        return (-2<BalancedUtil.BalFac(node)) and (BalancedUtil.BalFac(node)<2)